In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [2]:
filepath = Path("../sample_files/lvtetmesh_fibers_LDRB_1.vtk") # this file is too large. create one with "compute_fibers.ipynb"
lv = LV.from_pyvista_read(filepath)

In [3]:
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.APEX_BASE_REGIONS, overwrite=False)
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.EPI_ENDO, overwrite=False)
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.SURFS, overwrite=False)
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.AM_SURFS, overwrite=False)
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.SURFS_DETAILED, overwrite=False)

In [4]:
s = lv.create_surface_oi_from_surface(LV_MESH_DATA.SURFS_DETAILED)

In [5]:
lv.set_aortic_info()
lv.set_mitral_info()

In [6]:
aortic_bc = lv.create_spring_rim_bc("AORTIC_BC", LV_SURFS.BORDER_AORTIC)
mitral_bc = lv.create_spring_rim_bc("MITRAL_BC", LV_SURFS.BORDER_MITRAL)

In [7]:
from febio_python.feb import FEBio_feb
feb_template_path = Path("../sample_files/sample_lv_template.feb")
feb = FEBio_feb.from_file(feb_template_path)
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Boundary: 1
-> Loads: 1
-> Discrete: 2
-> LoadData: 3
-> Output: 1

In [29]:
lv._discrete_sets

{'AORTIC_BC': array([[  0,  74],
        [  1,  74],
        [  2,  75],
        ...,
        [889,  80],
        [890,  90],
        [891, 102]], dtype=int64),
 'MITRAL_BC': array([[  0, 120],
        [  1, 114],
        [  2, 115],
        ...,
        [793, 105],
        [794, 120],
        [795, 123]], dtype=int64)}

In [43]:
adjusted_discrete_sets = {}
adjustment = len(lv.nodes())+1
for key, values in lv._discrete_sets.items():
  adj_vals = np.copy(values)
  adj_vals[:, 1] += adjustment
  adjusted_discrete_sets[key] = adj_vals
  adjustment += len(adj_vals)+1

In [51]:
lv.get_bc("AORTIC_BC")[1]["RIM_NODES"]

array([[-22.78620119,   3.67571611,  34.03371496],
       [-22.08205709,   4.07914289,  34.83812434],
       [-21.32236098,   4.38547153,  35.63478681],
       [-20.51442915,   4.59175192,  36.41603008],
       [-19.6660424 ,   4.69599746,  37.17433036],
       [-18.78537119,   4.69720422,  37.90238479],
       [-17.88089684,   4.59536058,  38.59318182],
       [-16.96132994,   4.39144733,  39.24006869],
       [-16.03552643,   4.08742829,  39.83681553],
       [-15.1124023 ,   3.68623131,  40.37767534],
       [-14.20084774,   3.19172015,  40.85743934],
       [-13.30964151,   2.60865722,  41.27148714],
       [-12.44736642,   1.94265773,  41.61583125],
       [-11.62232666,   1.20013562,  41.88715542],
       [-10.8424678 ,   0.38824178,  42.08284668],
       [-10.11530031,  -0.48520479,  42.20102039],
       [ -9.44782721,  -1.41179235,  42.24053849],
       [ -8.84647663,  -2.38259733,  42.20102039],
       [ -8.31703992,  -3.38827036,  42.08284668],
       [ -7.86461582,  -4.41912

In [53]:
lv.nodes().shape

(149636, 3)

In [54]:
lv.get_bc("AORTIC_BC")[1]["RIM_NODES"].shape

(128, 3)

In [57]:
len(lv.nodes())+ len(lv.get_bc("MITRAL_BC")[1]["RIM_NODES"]) +1

149765

In [63]:
lv.mesh

UnstructuredGrid (0x19d8a9e7d60)
  N Cells:	734930
  N Points:	149636
  X Bounds:	-2.198e+01, 5.639e+01
  Y Bounds:	-3.899e+01, 4.336e+01
  Z Bounds:	-4.722e+01, 3.682e+01
  N Arrays:	23

In [59]:
from febio_python.feb import FEBio_feb
feb_template_path = Path("../sample_files/sample_lv_template.feb")
feb = FEBio_feb.from_file(feb_template_path)
feb.add_nodes([
  {"name": "LV", 
   "nodes": lv.nodes(),
   },
  ])
feb.add_nodes([
  {"name": "AORTIC_RIM", 
   "nodes": lv.get_bc("AORTIC_BC")[1]["RIM_NODES"]
   },
  ],initial_el_id=len(lv.nodes())+1)
feb.add_nodes([
  {"name": "AORTIC_RIM", 
   "nodes": lv.get_bc("MITRAL_BC")[1]["RIM_NODES"]
   },
  ],initial_el_id=len(lv.nodes())+ len(lv.get_bc("MITRAL_BC")[1]["RIM_NODES"]) +1)
feb.add_elements([
  {
    "name": "LV", 
    "type": "tet4",
    "mat": "1",
    "elems": lv.cells(VTK_ELEMENTS.TETRA)
    }
  ])
feb.add_nodesets({
  "ENDO": lv.get_nodeset(LV_SURFS.ENDO),
  "EPI": lv.get_nodeset(LV_SURFS.EPI) 
  })
feb.add_surfaces({
  "ENDO": lv.get_surface_oi(LV_SURFS.ENDO),
  })
feb.add_discretesets(adjusted_discrete_sets)
feb.add_meshdata([
  {
    "elem_set": "LV", 
    "var": "mat_axis",
    "elems": {
      "a": lv.get(GEO_DATA.MESH_CELL_DATA, LV_FIBERS.F0), #data['MESH_ELEM_DATA']["LV_FIBERS"],
      "d": lv.get(GEO_DATA.MESH_CELL_DATA, LV_FIBERS.S0),
      }
    }
  ])
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Geometry: 9
--> Nodes 'LV': 149636
--> Nodes 'AORTIC_RIM': 128
--> Nodes 'AORTIC_RIM': 128
--> Elements 'LV': 734930
-> Boundary: 1
-> Loads: 1
-> Discrete: 2
-> LoadData: 3
-> MeshData: 1
-> Output: 1

In [60]:
feb.loads().find("surface_load").attrib["surface"] = "ENDO"

In [61]:
discrete = feb.discrete()
discrete.find("discrete").attrib["discrete_set"] = "AORTIC_BC"
import xml.etree.ElementTree as ET
subel = ET.SubElement(discrete, 'discrete')
subel.set("discrete_set", "MITRAL_BC")
subel.set("dmat", "1")

In [62]:
feb.write("../sample_files/sample_lv_template_mod.feb")